In [ ]:
#call workspace from config file in another environment
from azureml.core import Workspace
ws = Workspace.from_config(path="./.azureml/ws_config.json")

In [ ]:
ws.get_details()

In [ ]:
%%writefile my_sklearn_lr.py
#Remember our linear regression example? Let's train it locally and register it

#prepare the environment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sklearn
#load the data
from sklearn.datasets import load_boston
boston_dataset = load_boston()


#prepare the data
from sklearn.model_selection import train_test_split
num_Rooms_Train, num_Rooms_Test, med_price_Train, med_Price_Test = train_test_split(boston_dataset.data[:,5].reshape(-1,1), boston_dataset.target.reshape(-1,1))


#implement linear regression model
from sklearn.linear_model import LinearRegression
price_room = LinearRegression()
price_room.fit (num_Rooms_Train,med_price_Train)

# Let's save this model fit out
from sklearn.externals import joblib
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=price_room, filename="outputs/bh_lr.pkl")


In [ ]:
#!ls -al
!cat my_sklearn_lr.py

In [ ]:
#Let's register this model with our workspace

from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="bh_lr.pkl", model_name="boston_housing_lr")

In [ ]:
mymodel = Model(workspace=ws, name="boston_housing_lr")
mymodel.download(exist_ok=True)

In [ ]:
from sklearn.externals import joblib
localmodel = joblib.load("bh_lr.pkl")

In [ ]:
!ls -al 


In [ ]:
mymodel.delete()

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)

In [ ]:
print(list_vms)

In [ ]:
compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "Standard_DS1_v2"

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.20.3")
dependencies.add_pip_package("numpy==1.16.2")
dependencies.add_pip_package("pandas==0.23.4")
dependencies.add_pip_package("matplotlib==3.0.0")
compute_config.environment.python.conda_dependencies = dependencies

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=".",script="bh_sklearn_lr.py", run_config=compute_config)
experiment = Experiment(workspace=ws, name="lq_simple_model")
run = experiment.submit(config=script_run_config)
run.wait_for_completion(show_output=True)

In [ ]:
run.get_file_names()

In [ ]:
run.download_file(name="outputs/bh_lr.pkl")

In [ ]:
!ls

In [1]:
ls -al

 Volume in drive C is OSDisk
 Volume Serial Number is 90E5-5ABE

 Directory of C:\Users\venkati.naidu\hello\Coursera



File Not Found


In [ ]:
from sklearn.externals import joblib
import numpy as np

run_model = joblib.load('bh_lr.pkl')
run_model.predict(np.array([7.354]).reshape(-1,1))

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="bh_lr.pkl", model_name="lq_simple_train")

In [ ]:
run.register_model(model_name="sklearn_bh_lr", model_path="outputs/bh_lr.pkl")